In [ ]:
import cv2
from PIL import Image
from models.face_classifier_googlenet import FaceGoogLeNet
import torch
from ultralytics import YOLO
from utils.system import get_available_device
from collections import defaultdict
import torch.nn.functional as F
import os


device = get_available_device()

KNOWN_FACES_PATH='./known_faces'
CLASSIFIER_MODEL_WEIGHTS_PATH = './best_models/googlenet_face_classifier_05-04_09_04_epoch_2.pt'

# Model
detector_model = YOLO("./best_models/yolo_decoder_best.pt").to(device)

classifier_model = FaceGoogLeNet(embedding_size=128, pretrained=False).to(device)
load_parameters = torch.load(CLASSIFIER_MODEL_WEIGHTS_PATH, weights_only=False, map_location=torch.device(device))
classifier_model.load_state_dict(load_parameters['state_model'])

GPU is not available, using CPU instead
Using device: cpu


/Users/user/Documents/university/Дипломы/Никита/project/venv_yolo/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/user/Documents/university/Дипломы/Никита/project/venv_yolo/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/user/Documents/university/Дипломы/Никита/project/venv_yolo/lib/python3.12/site-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
 

FileNotFoundError: [Errno 2] No such file or directory: './best_models/googlenet_face_classifier_05-04_09_04_epoch_2'

In [7]:
known_embeddings = defaultdict(list)

In [8]:
from torchvision.transforms import v2

transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.uint8, scale=True),
    v2.Resize((160, 160)),  # Измените размер под вашу модель
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])


def compare_embeddings(embedding, threshold=0.7):
    best_match = None
    min_distance = float('inf')

    for name, known_person_embeddings in known_embeddings.items():
        for embeddings in known_person_embeddings:
            distance = F.pairwise_distance(embedding, embeddings)

            if distance < min_distance:
                min_distance = distance
                best_match = name

    if min_distance <= threshold:
        return best_match
    else:
        return None
    

def detect_faces(image):
    with torch.no_grad():  # Отключите вычисление градиентов во время инференса
        result = detector_model([image])[0]
    
    return result.boxes.xyxy
    
        
def recognize_faces(image, boxes):
    cropped_faces = [
        transform(image.crop(list(map(float,bbox))))
        for bbox in boxes
    ]
    if cropped_faces:  # Убедитесь, что список не пустой
        face_tensors = torch.stack(cropped_faces)
        embeddings = classifier_model(face_tensors)
    else:
        embeddings = torch.empty(0)
        
    return embeddings

In [9]:
from utils.system import get_last_dirname


for path_dir, dir_list, file_list in os.walk(KNOWN_FACES_PATH):
    for file_path in file_list:
        image_path = os.path.join(path_dir, file_path)
        image = Image.open(image_path)
        
        boxes = detect_faces(image)
        embeddings = recognize_faces(image, boxes)
        
        name = get_last_dirname(path_dir)
        known_embeddings[name].append(embeddings[0])


0: 448x640 1 face, 77.9ms
Speed: 10.5ms preprocess, 77.9ms inference, 3.7ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 face, 61.2ms
Speed: 3.1ms preprocess, 61.2ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 face, 64.0ms
Speed: 3.5ms preprocess, 64.0ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 face, 67.1ms
Speed: 3.9ms preprocess, 67.1ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 face, 62.6ms
Speed: 3.4ms preprocess, 62.6ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 face, 65.2ms
Speed: 2.5ms preprocess, 65.2ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 face, 60.3ms
Speed: 3.0ms preprocess, 60.3ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 face, 59.7ms
Speed: 2.5ms preprocess, 59.7ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)


In [10]:
import cv2
import time


# 2.  Захват видео с веб-камеры
cap = cv2.VideoCapture(0)  # 0 - индекс веб-камеры по умолчанию. Измените, если у вас несколько камер.


# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    raise IOError("Не удается открыть веб-камеру")


frame_count = 0
start_time = time.time()
fps = 0

# 3. Основной цикл обработки кадров
while(True):
    # Считываем кадр с веб-камеры
    ret, frame = cap.read()

    # Если кадр не был успешно считан, выходим из цикла
    if not ret:
        break
    
    frame_count += 1
    elapsed_time = time.time() - start_time
    
    
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)
    
    boxes = detect_faces(pil_image)
    embeddings = recognize_faces(pil_image, boxes)
    
    for bbox, embedding in zip(boxes, embeddings):
        x1 = int(bbox[0])
        y1 = int(bbox[1])
        x2 = int(bbox[2])
        y2 = int(bbox[3])
        
        name = compare_embeddings(embedding) or 'Unknown'

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    

    if elapsed_time > 1:  # Обновляем FPS каждую секунду
        fps = frame_count / elapsed_time
        start_time = time.time()
        frame_count = 0

    # Добавим FPS на кадр для отображения на экране
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # 7. Отображение результата
    cv2.imshow('Обнаружение лиц', frame)

    # 8. Выход из цикла при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

# 9. Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 face, 54.7ms
Speed: 6.3ms preprocess, 54.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 51.0ms
Speed: 2.4ms preprocess, 51.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 52.9ms
Speed: 2.0ms preprocess, 52.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 53.3ms
Speed: 2.0ms preprocess, 53.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 50.3ms
Speed: 1.8ms preprocess, 50.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 50.2ms
Speed: 1.8ms preprocess, 50.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 60.1ms
Speed: 1.7ms preprocess, 60.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 58.3ms
Speed: 2.0ms preprocess, 58.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x

KeyboardInterrupt: 